- Remember to include `self` as the first parameter to allow the method to access and operate on the instance's attributes and methods.


In [41]:
class Percolation:
    def __init__(self, n):
        """ creates n-by-n grid, with all sites initially blocked """
        if n < 0:
            raise ValueError('n should be a non-negative number')
        self.n = n
        self.grid = [[False] * n for _ in range(n)]
        # Additional data structure to keeo track of the connectivity of sites
        # Quick-Find data structure assigns the very root node
        # Initialize each node is its own root. Add 2 for virtual top and bottom sites
        self.id = [i for i in range(n * n + 2)]
    
    def open(self, row, col):
        """ opens the site (row, col) if it is not open already """
        if not self.is_valid(row, col):
            raise ValueError('row or column is out of bound')
        site = self.map_to_id(row, col)
        if not self.grid[row - 1][col - 1]:
            self.grid[row - 1][col - 1] = True

        if 1 <= site & site <= n:
            self.union(site, 0)
        if n * (n - 1) <= site & site <= n * n:
            self.union(site, n * n + 1)
    
        neighbors = [
            (row - 1, col), (row + 1, col), (row, col - 1), (row, col + 1)
        ]
        for r, c in neighbors:
            if self.is_valid(r, c) and self.is_open(r, c):
                neighbor_site = self.map_to_id(r, c)
                self.union(site, neighbor_site)

    def is_open(self, row, col):
        """ is the site (row, col) open? """
        if not self.is_valid(row, col):
            raise ValueError('row or column is out of bound')
        return self.grid[row - 1][col - 1]

    def is_full(self, row, col):
        """ is the site (row, col) full? """
        if not self.is_valid(row, col):
            raise ValueError('row or column is out of bound')
        site = self.map_to_id(row, col)
        return self.find(site) == self.find(0)
        
    def number_of_open_sites(self):
        """returns the number of open sites """
        return sum(row.count(True) for row in self.grid)

    def percolates(self):
        """ does the system percolate? """
        return self.find(0) == self.find(self.n * self.n + 1)

    def is_valid(self, row, col):
        """ checks if row and column is not out of bounds """
        return 0 < row <= self.n and 0 < col <= self.n
    
    def map_to_id(self, row, col):
        return (row - 1) * self.n + col
    
    # Quick Union
    def union(self, p, q):
        pid = self.find(p)
        qid = self.find(q)
        if pid != qid:
            self.id[pid] = qid
    
    def find(self, p):
        while p != self.id[p]:
            p = self.id[p]
        return p



In [49]:
n = 3;
p = Percolation(n);

p.open(1, 1)
p.open(1, 2)
p.open(1, 3)
p.open(2, 1)
p.open(2, 2)
p.open(3, 2)
p.open(3, 1)

print("Is (1, 1) full? %s" % p.is_open(1, 1))
print("Is (1, 1) full? %s" % p.is_full(1, 1))
print("Is (3, 2) full? %s" % p.is_full(3, 2))
print("Does the system percolate? %s" % p.percolates())

Is (1, 1) full? True
Is (1, 1) full? True
Is (3, 2) full? True
Does the system percolate? True


In [23]:
x =  Percolation(5)
import random

random.randint(1, 5)

ls = [2, 3, 4]
[(x - 2)**2 for x in ls]

[0, 1, 4]

In [36]:
import random
import time

class PercolationStats:
    def __init__(self, n, trials) -> None:
        """ perform independent trials on an n-by-n grid """
        if n <= 0 or trials <= 0:
            raise ValueError("Invalid input. Both n and trials must be greater than 0.")
        self.n = n
        self.trials = trials
        self.fraction = []
        for i in range(trials):
            p = Percolation(n)
            while not p.percolates():
                row = random.randint(1, n)
                col = random.randint(1, n)
                p.open(row, col)
            fraction = p.number_of_open_sites() / (n * n)
            self.fraction.append(fraction)
    
    def mean(self):
        """ sample mean of percolation threshold """
        result = sum(self.fraction) / self.trials
        self.mean = result
        return result
    
    def stddev(self):
        """ sample standard deviation of percolation threshold """
        result = sum([(x - self.mean)**2 for x in self.fraction]) / (self.trials - 1)
        self.stddev = result
        return result
    
    def confidenceLo(self):
        """ low endpoint of 95% confidence interval """
        return self.mean - 1.96 * self.stddev / (self.trials ** 0.5)
    
    def confidenceHi(self):
        """ high endpoint of 95% confidence interval """
        return self.mean + 1.96 * self.stddev / (self.trials ** 0.5)

stats = PercolationStats(4, 5)
print(stats.mean(), stats.stddev(), stats.confidenceLo(), stats.confidenceHi())
    


KeyboardInterrupt: 